In [1]:
# break condition, not filled and possibilities are 0
from typing import List

In [2]:
import string
import collections
NINE = 9

def _sqidx1(i, j):
    return (i // 3) * 3 + j // 3
def _sqidx2(i, j):
    return (i % 3) * 3 + (j % 3)

class Board():
    digits = set([c for c in string.digits if c != "0"])
    def __init__(self, init_state: List[List[str]]):
        self.board = [
                init_state[i//NINE][i%NINE]
                for i in range(NINE * NINE)]
        self.sqridx = {i: [] for i in range(NINE)}
        self.colidx = {i: [] for i in range(NINE)}
        self.rowidx = {i: [] for i in range(NINE)}
        for i in range(NINE):
            for j in range(NINE):
                a = _sqidx1(i, j)
                self.sqridx[a].append(i * NINE + j)
                self.rowidx[i].append(i * NINE + j)
                self.colidx[j].append(i * NINE + j)
        self.rows = [set([]) for j in range(NINE)]
        self.cols = [set([]) for j in range(NINE)]
        self.squares = [set([]) for j in range(NINE)]
        self.poss = [set([]) for i in range(NINE * NINE)]
        self.unk = sum([c == "." for c in self.board])
        self.update()
        
    def get_board(self):
        return [[self.board[i * NINE + j] 
                for j in range(NINE)]
               for i in range(NINE)]

    def update(self):
        for i in range(NINE):
            self.rows[i] = (self.digits 
                    - set([self.board[w] for w in self.rowidx[i]]))
            self.cols[i] = (self.digits 
                    - set([self.board[w] for w in self.colidx[i]]))
            self.squares[i] = (self.digits 
                    - set([self.board[w] for w in self.sqridx[i]]))
        for i in range(NINE * NINE):
            self.poss[i] = (self.rows[i // NINE] 
                    & self.cols[i % NINE]
                    & self.squares[_sqidx1(i // NINE, i % NINE)])
            
    def insert(self, i, val):
        self.board[i] = val
        self.update()
        self.unk -= 1
        
    def detect_only_possibility(self):
        ret = None
        for i in range(NINE):
            cnt = collections.Counter()
            for j in range(NINE):
                cnt = cnt + collections.Counter(self.poss[i][j])
            for key, val in cnt.items():
                if val == 1:
                    print("a")
                    for j in range(NINE):
                        if key in self.poss[i][j]:
                            self.insert(i, j, key)
                            
            cnt = collections.Counter()
            for j in range(NINE):
                cnt = cnt + collections.Counter(self.poss[j][i])
            for key, val in cnt.items():
                if val == 1:
                    print("b")
                    for j in range(NINE):
                        if key in self.poss[j][i]:
                            self.insert(j, i, key)
                            
            cnt = collections.Counter()
            sq = self.sqridx[i]
            for pair in sq:
                cnt = cnt + collections.Counter(
                    self.poss[pair[0]][pair[1]])
            for key, val in cnt.items():
                if val == 1:
                    print("c")
                    for pair in sq:
                        if key in self.poss[pair[0]][pair[1]]:
                            self.insert(pair[0], pair[1], key)
            
    def find_min(self):
        mn = NINE
        idx = -1
        for i in range(NINE * NINE):
            if self.board[i] == ".":
                v = len(self.poss[i])
                if v < mn:
                    mn = v
                    idx = i
        return mn, idx
            


        
class Solution:
    def solveSudoku(self, board: List[List[str]]) -> None:
        """
        Do not return anything, modify board in-place instead.
        """
        sudoku = Board(board)
        sudoku.update()
        sudoku = _recur(sudoku)
        arr = sudoku.board
        for i in range(NINE):
            for j in range(NINE):
                board[i][j] = arr[i * NINE + j]
        
        
def _recur(sudoku):
    cont = True
    while cont and (sudoku.unk > 0):            
        mn, idx = sudoku.find_min()
        if mn == 1:
            sudoku.insert(idx, list(sudoku.poss[idx])[0])
        elif mn == 0:
            cont = False
            sudoku = None
        else:
            cont = False
            lst = list(sudoku.poss[idx])
            for i in range(mn):
                arr = sudoku.get_board()
                arr[idx // NINE][idx % NINE] = lst[i]
                v = Board(arr)
                v = _recur(v)
                if (v is not None) and (v.unk == 0):
                    sudoku = v
                    break
    return sudoku

In [3]:
board = [["5","3",".",".","7",".",".",".","."],["6",".",".","1","9","5",".",".","."],[".","9","8",".",".",".",".","6","."],["8",".",".",".","6",".",".",".","3"],["4",".",".","8",".","3",".",".","1"],["7",".",".",".","2",".",".",".","6"],[".","6",".",".",".",".","2","8","."],[".",".",".","4","1","9",".",".","5"],[".",".",".",".","8",".",".","7","9"]]

In [4]:
board = [[".",".","9","7","4","8",".",".","."],["7",".",".",".",".",".",".",".","."],[".","2",".","1",".","9",".",".","."],[".",".","7",".",".",".","2","4","."],[".","6","4",".","1",".","5","9","."],[".","9","8",".",".",".","3",".","."],[".",".",".","8",".","3",".","2","."],[".",".",".",".",".",".",".",".","6"],[".",".",".","2","7","5","9",".","."]]

In [5]:
Solution().solveSudoku(board)

In [6]:
board

[['5', '1', '9', '7', '4', '8', '6', '3', '2'],
 ['7', '8', '3', '6', '5', '2', '4', '1', '9'],
 ['4', '2', '6', '1', '3', '9', '8', '7', '5'],
 ['3', '5', '7', '9', '8', '6', '2', '4', '1'],
 ['2', '6', '4', '3', '1', '7', '5', '9', '8'],
 ['1', '9', '8', '5', '2', '4', '3', '6', '7'],
 ['9', '7', '5', '8', '6', '3', '1', '2', '4'],
 ['8', '3', '2', '4', '9', '1', '7', '5', '6'],
 ['6', '4', '1', '2', '7', '5', '9', '8', '3']]

In [12]:
class Solution:
    def minimumMountainRemovals(self, nums: List[int]) -> int:
        n = len(nums)
        right_decr = [0 for i in range(n)]
        left_incr = [0 for i in range(n)]
        p = n - 1
        while p > -1:
            val = nums[p]
            p2 = p + 1
            mx = 0
            while (p2 < n):
                if (val > nums[p2]) and (right_decr[p2] >= mx):
                    mx = right_decr[p2] + 1
                p2 += 1
            right_decr[p] = mx
            p -= 1
        p = 0
        while p < n:
            val = nums[p]
            p2 = p - 1
            mx = 0
            while (p2 > -1):
                if (val > nums[p2]) and (left_incr[p2] >= mx):
                    mx = left_incr[p2] + 1
                p2 -= 1
            left_incr[p] = mx
            p += 1
                    
        print(right_decr, left_incr)
        mx = 0
        for i in range(1, n-1):
            a = right_decr[i]
            b = left_incr[i]
            if (a > 0) and (b > 0) and (a + b > mx):
                mx = a + b
        return n - (mx + 1)

In [13]:
arr = [1, 3, 1]
Solution().minimumMountainRemovals(arr)

[0, 1, 0] [0, 1, 0]


0